In [40]:
#Multi-class classification
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim


In [41]:
torch.manual_seed(1)

In [42]:
z = torch.FloatTensor([1, 2, 3])
F.softmax(z, dim=0)

tensor([0.0900, 0.2447, 0.6652])

In [43]:
z = torch.rand(3, 5, requires_grad=True)

hypothesis = F.softmax(z, dim=1)
hypothesis

tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward0>)

In [44]:
y = torch.randint(5, (3,)).long()
y

tensor([0, 2, 1])

In [45]:
y_one_hot =  torch.zeros_like(hypothesis)
y_one_hot.scatter_(1, y.unsqueeze(1), 1)
y_one_hot

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])

In [46]:
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
cost

tensor(1.4689, grad_fn=<MeanBackward0>)

In [47]:
#비용 함수 메소드로 구하기
F.log_softmax(z, dim=0) #==torch.log(F.softmax(z, dim=1))


tensor([[-1.0753, -1.2247, -1.3200, -0.9157, -1.3987],
        [-1.0331, -1.1069, -0.9687, -1.0809, -0.9892],
        [-1.1943, -0.9793, -1.0405, -1.3452, -0.9644]],
       grad_fn=<LogSoftmaxBackward0>)

In [48]:
F.nll_loss(F.log_softmax(z, dim=1), y)

tensor(1.4689, grad_fn=<NllLossBackward0>)

In [49]:
F.cross_entropy(z, y) #z는 모델의 예측 결과값, y는 정답

tensor(1.4689, grad_fn=<NllLossBackward0>)

In [50]:
#소프트맥스 회귀 구현
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)
#클래스 3개

In [56]:
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(3, requires_grad=True)

optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 300000
for epoch in range(nb_epochs + 1):

    # Cost 계산
    z = x_train.matmul(W) + b #모델의 예측값 계산
    cost = F.cross_entropy(z, y_train)
    if epoch == 200000:
        optimizer = optim.SGD([W, b], lr=0.001)#뒤로갈수록 lr 감소시키기

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/300000 Cost: 1.098612
Epoch  100/300000 Cost: 0.704199
Epoch  200/300000 Cost: 0.622999
Epoch  300/300000 Cost: 0.565717
Epoch  400/300000 Cost: 0.515291
Epoch  500/300000 Cost: 0.467662
Epoch  600/300000 Cost: 0.421278
Epoch  700/300000 Cost: 0.375402
Epoch  800/300000 Cost: 0.329766
Epoch  900/300000 Cost: 0.285073
Epoch 1000/300000 Cost: 0.248155
Epoch 1100/300000 Cost: 0.232676
Epoch 1200/300000 Cost: 0.221399
Epoch 1300/300000 Cost: 0.211129
Epoch 1400/300000 Cost: 0.201736
Epoch 1500/300000 Cost: 0.193113
Epoch 1600/300000 Cost: 0.185170
Epoch 1700/300000 Cost: 0.177829
Epoch 1800/300000 Cost: 0.171027
Epoch 1900/300000 Cost: 0.164707
Epoch 2000/300000 Cost: 0.158821
Epoch 2100/300000 Cost: 0.153327
Epoch 2200/300000 Cost: 0.148186
Epoch 2300/300000 Cost: 0.143368
Epoch 2400/300000 Cost: 0.138843
Epoch 2500/300000 Cost: 0.134586
Epoch 2600/300000 Cost: 0.130574
Epoch 2700/300000 Cost: 0.126787
Epoch 2800/300000 Cost: 0.123206
Epoch 2900/300000 Cost: 0.119817
Epoch 3000

In [52]:
for i in range(8):
    print(F.softmax(x_train[i].matmul(W)+b, dim=0).argmax(), y_train[i])

tensor(2) tensor(2)
tensor(2) tensor(2)
tensor(2) tensor(2)
tensor(1) tensor(1)
tensor(1) tensor(1)
tensor(1) tensor(1)
tensor(0) tensor(0)
tensor(0) tensor(0)


In [53]:
#Module로 구현

model = nn.Linear(4, 3)

optimizer = optim.SGD(model.parameters(), lr=0.01)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.cross_entropy(prediction, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 20번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.849513
Epoch  100/1000 Cost: 0.839934
Epoch  200/1000 Cost: 0.757753
Epoch  300/1000 Cost: 0.704903
Epoch  400/1000 Cost: 0.665221
Epoch  500/1000 Cost: 0.633748
Epoch  600/1000 Cost: 0.608009
Epoch  700/1000 Cost: 0.586476
Epoch  800/1000 Cost: 0.568119
Epoch  900/1000 Cost: 0.552216
Epoch 1000/1000 Cost: 0.538244
